In [1]:
import pickle, os
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict
from ast import literal_eval
import json
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from pathlib import Path

In [2]:
folder = [Path(".\\asd_result\\train"),Path(".\\asd_result\\test"), Path(".\\asd_result\\val")]
subfolder=[]
for i in folder:
    temp=[i / f for f in os.listdir(i) if not f.startswith('.') and os.path.isdir(i / f)]
    subfolder = subfolder + temp
print(subfolder)

[WindowsPath('asd_result/train/train_vimeo_410316487'), WindowsPath('asd_result/train/train_vimeo_420368971'), WindowsPath('asd_result/train/train_vimeo_429747058'), WindowsPath('asd_result/train/train_vimeo_440357459'), WindowsPath('asd_result/train/train_vimeo_460704945'), WindowsPath('asd_result/train/train_vimeo_471034832'), WindowsPath('asd_result/train/train_vimeo_480532944'), WindowsPath('asd_result/train/train_vimeo_481373806'), WindowsPath('asd_result/train/train_vimeo_491282558'), WindowsPath('asd_result/train/train_vimeo_503562591'), WindowsPath('asd_result/train/train_vimeo_526348428'), WindowsPath('asd_result/train/train_vimeo_543689289'), WindowsPath('asd_result/train/train_vimeo_546221378'), WindowsPath('asd_result/train/train_vimeo_562725349'), WindowsPath('asd_result/train/train_vimeo_611083383'), WindowsPath('asd_result/train/train_vimeo_656699672'), WindowsPath('asd_result/test/test_vimeo_414944373'), WindowsPath('asd_result/test/test_vimeo_425295077'), WindowsPath('

In [3]:
# just want to test on a single file
# subfolder=subfolder[-3:-2]
# print(subfolder)

load ASD result and save to ASDout.csv that is of ['frame', 'scene_id', 'track_id', ASDscore', 'bbox_xmin', 'bbox_y_min', 'bbox_xmax', 'bbox_ymax']

In [4]:
output_file = 'ASDout.csv'

for i in subfolder:
    path = i / 'pywork'    
    filename=['faces.pckl', 'scene.pckl', 'scores.pckl', 'tracks.pckl']
    with open(path / filename[3], 'rb') as f:
        data = pickle.load(f)
    with open(path / filename[2], 'rb') as f:
        scoredata = pickle.load(f)
    with open(path / filename[1], 'rb') as f:
        scenedata = pickle.load(f)
        
    scene_id=[]
    for j in range(len(scenedata)):
        id = int(str(scenedata[j]).split(",")[0].split("=")[-1])
        scene_id.append(id) # scene_id is the starting frame of each scene
#     print(scene_id)
   
    d,df=[],[]
    scene=-1
    for j in range(len(data)):
        # if the start frame of a track == the start frame of a scene, the scene_id+=1
#         print("track:",i,"start frame:",data[i]['track']['frame'][0])
        while scene+1 < len(scene_id) and data[j]['track']['frame'][0] >= scene_id[scene+1]:
            scene+=1
#             print("scene is now", scene)
        d.append({'frame':data[j]['track']['frame'], 
                  'scene_id':scene,
                  'track_id':j,
                  'ASDscore':np.append(0, scoredata[j]).round(decimals=1), # len(scoredata) = len(data)-1, so add 0 at the head
                  'bbox_xmin':data[j]['track']['bbox'][:,0].astype(int), 
                  'bbox_ymin':data[j]['track']['bbox'][:,1].astype(int), 
                  'bbox_xmax':data[j]['track']['bbox'][:,2].astype(int),
                  'bbox_ymax':data[j]['track']['bbox'][:,3].astype(int)})
        df.append(pd.DataFrame(data=d[j]))
        
    result = pd.concat(df)
    result_sort=result.sort_values(by=["frame", "track_id"])
    result_sort.to_csv(i / output_file, index=False)

Load ASDout.csv and save to turn.csv [frame, [speaker_id], [listener_id]]

In [5]:
input_file = 'ASDout.csv'
output_file = "turn.csv"

for i in subfolder:
    df = pd.read_csv(i / input_file)
    # select rows of the same frame
    # define turn_speaker = [track_id1, track_id2, ...] for all the track_id whose score>0, or =[] if no track_id has score>0
    # make ["frame" "speaker" "listener"] as a new dataframe and save to .csv
    frame_id, speaker, listener = [],[],[]
    frame_id = list(df['frame'].unique())
    spk,lsn = [],[]
    for j in range(len(df)):
        if j>0 and df['frame'].iloc[j]!=df['frame'].iloc[j-1]:
            speaker.append(spk)
            listener.append(lsn)
            spk,lsn = [],[]
        if df['ASDscore'].iloc[j]>0:
            spk.append(df['track_id'].iloc[j])
        else:
            lsn.append(df['track_id'].iloc[j])
        if j==len(df)-1:
            speaker.append(spk)
            listener.append(lsn)
    pd.DataFrame({'frame':frame_id, 'speaker':speaker, 'listener':listener}).to_csv(i / output_file, index=False)               

Remove short silence intervals  

In [6]:
# def commonelems(x,y):
#     common=[]
#     for value in x:
#         if value in y:
#             common.append(value)
#     return common

def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = [value]
    elif type(dictionary[key]) == list:
        dictionary[key].append(value)
    else:
        dictionary[key] = [dictionary[key], value]

In [7]:
PP1 = True # preprocessing 1, remove short silence intervals
PP1_th = 25 # threshold to determine short silence interval
# here 25 is 1sec when fps=25, so silence intervals that <1s will be treatedd as short silence
input_file = "turn.csv"
output_file = "turn_PP1.csv"

if PP1:
    for i in subfolder:

        df=pd.read_csv(i / input_file)
        speaker = df['speaker'].apply(lambda x: literal_eval(str(x))).tolist()
        listener = df['listener'].apply(lambda x: literal_eval(str(x))).tolist()
        frame_id = df['frame'].tolist()

        d_right, d_left={},{}
        for j in range(1, len(frame_id)):
            right = list(set(speaker[j-1])-set(speaker[j]))
            for item in right:
                add2dict(d_right, key=item, value=j-1)
            left = list(set(speaker[j])-set(speaker[j-1]))
            for item in left:
                add2dict(d_left, key=item, value=j)
            # d_left is {speaker_id:[left endpoint]}, d_right is {speaker_id:[right endpoint]}

        d={}
        for key in d_right:
            if key in d_left:
                if d_right[key][0]<d_left[key][0]:
                    d[key] = list(zip(d_right[key], d_left[key]))
                elif len(d_left[key])>1 and d_right[key][0]<d_left[key][1]:
                    d[key] = list(zip(d_right[key], d_left[key][1:]))
        # now d is {speaker_id:[(right-endpoint, left-endpoint),(),...]}
        # we just need to check the length of each left-right, which is the silence interval, and fill the short ones
        for key in d:
            for right, left in d[key]:
                if left-right-1 < PP1_th:
                    for j in range(right+1, left):
                        speaker[j]+=[key]
                        if key in listener[j]:
                            listener[j].remove(key) 
        df = pd.DataFrame({'frame':frame_id, 'speaker':speaker, 'listener':listener}, dtype = int)
        df.to_csv(i / output_file, index=False)

Remove short utterance intervals

In [8]:
PP2 = True # preprocessing 2, remove short utterance intervals
PP2_th = 25 # threshold to determine short utterance interval
input_file = "turn_PP1.csv"
output_file = "turn_PP2.csv"

if PP2:
    for i in subfolder:

        df=pd.read_csv(i / input_file)
        speaker = df['speaker'].apply(lambda x: literal_eval(str(x))).tolist()
        listener = df['listener'].apply(lambda x: literal_eval(str(x))).tolist()
        frame_id = df['frame'].tolist()

        d_right, d_left={},{}
        for j in range(1, len(frame_id)):
            right = list(set(speaker[j-1])-set(speaker[j]))
            for item in right:
                add2dict(d_right, key=item, value=j-1)
            left = list(set(speaker[j])-set(speaker[j-1]))
            for item in left:
                add2dict(d_left, key=item, value=j)
            # d_left is {speaker_id:[left endpoint]}, d_right is {speaker_id:[right endpoint]}

        d={}
        for key in d_right:
            if key in d_left:
                if d_left[key][0]<=d_right[key][0]:
                    d[key] = list(zip(d_left[key], d_right[key]))
                elif len(d_right[key])>1 and d_left[key][0]<=d_right[key][1]:
                    d[key] = list(zip(d_left[key], d_right[key][1:]))
        # now d is {speaker_id:[(left-endpoint, right-endpoint),(),...]}
        # we just need to check the length of each right-left, which is the utterance interval, and remove the short ones
        for key in d:
            for left, right in d[key]:
                if right+1-left < PP2_th:
                    for j in range(left, right+1):
                        if key in speaker[j]:
                            speaker[j].remove(key)
                        listener[j]+=[key]
        df = pd.DataFrame({'frame':frame_id, 'speaker':speaker, 'listener':listener}, dtype = int)
        df.to_csv(i / output_file, index=False)

get training sample information (x_id, x_frame, y_id)

In [9]:
PP3 = True
LEN_th_max = 10*25 # here 10*25 means 10 seconds (25fps)
LEN_th_min = 25 # the potential sampling range of x before y_start
input_file = "turn_PP2.csv"
output_file = "turn.json"
# output (x_id, x_start, x_end, y_id, y_start, y_end), which is necessary information for a training sample

if PP3:
    for i in subfolder:   # find the position of x,y
        df=pd.read_csv(i / input_file)
        speaker = df['speaker'].apply(lambda x: literal_eval(str(x))).tolist()
        listener = df['listener'].apply(lambda x: literal_eval(str(x))).tolist()
        frame_id = df['frame'].tolist()
#         d = dict(zip(frame_id, zip(speaker, listener))    
#         df = pd.DataFrame(speaker, index=frame_id, columns =['speaker_id'])
        
        d_right, d_left={},{}
        for j in range(1, len(frame_id)):
            right = list(set(speaker[j-1])-set(speaker[j]))
            for item in right:
                add2dict(d_right, key=item, value=j-1)
            left = list(set(speaker[j])-set(speaker[j-1]))
            for item in left:
                add2dict(d_left, key=item, value=j)
            # d_left is {speaker_id:[left endpoint]}, d_right is {speaker_id:[right endpoint]}
                 
        d={}
        for key in d_right:
            if key in d_left:
                if d_left[key][0]<=d_right[key][0]:
                    d[key] = list(zip(d_left[key], d_right[key]))
                elif len(d_right[key])>1 and d_left[key][0]<=d_right[key][1]:
                    d[key] = list(zip(d_left[key], d_right[key][1:]))
        # now d is {speaker_id:[(left-endpoint, right-endpoint),(),...]}
        # we can assign y_id=speaker_id, y_start=left_endpoint, y_end=right_endpoint, 
        # x_start=left_endpoint-LEN_th, x_end=left_endpoint-1, x_id=listener in [x_start:x_end+1],
        training_sample=[]
        for key in d:
            for left, right in d[key]:
                y_id = key
                y_start = left # this is index, not frame number, the same below. 
                y_end = right
                x_start = left-LEN_th_max if left-LEN_th_max >= 0 else 0
                x_end = left-1 if left-1 >= 0 else 0
                x_id = list(set().union(*listener[x_start:x_end+1]))
                for x in x_id:
                    x_mask = [x in j for j in listener[x_start:x_end+1]]*np.arange(x_start,x_end+1)
                    x_frame = x_mask[x_mask!=0]
                    x_frame = np.array(frame_id)[x_frame] # convert to frame number
                    if len(x_frame) >= LEN_th_min:
                        training_sample.append({"x_id":x, "x_frame":list(map(int, x_frame)), "y_id":y_id})
                 
        with open(i / output_file,'w') as fp:
            json.dump(training_sample, fp) 
        print("finish "+str(i))

finish asd_result\train\train_vimeo_410316487
finish asd_result\train\train_vimeo_420368971
finish asd_result\train\train_vimeo_429747058
finish asd_result\train\train_vimeo_440357459
finish asd_result\train\train_vimeo_460704945
finish asd_result\train\train_vimeo_471034832
finish asd_result\train\train_vimeo_480532944
finish asd_result\train\train_vimeo_481373806
finish asd_result\train\train_vimeo_491282558
finish asd_result\train\train_vimeo_503562591
finish asd_result\train\train_vimeo_526348428
finish asd_result\train\train_vimeo_543689289
finish asd_result\train\train_vimeo_546221378
finish asd_result\train\train_vimeo_562725349
finish asd_result\train\train_vimeo_611083383
finish asd_result\train\train_vimeo_656699672
finish asd_result\test\test_vimeo_414944373
finish asd_result\test\test_vimeo_425295077
finish asd_result\test\test_vimeo_472545103
finish asd_result\test\test_vimeo_492548673
finish asd_result\test\test_vimeo_563666778
finish asd_result\test\test_vimeo_650895335


check feature availability for each training sample

In [10]:
def bb_intersection_over_union(boxA, boxB, evalCol = False):
	# CPU: IOU Function to calculate overlap between two image
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	interArea = max(0, xB - xA) * max(0, yB - yA)
	boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
	boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
	if evalCol == True:
		iou = interArea / float(boxAArea)
	else:
		iou = interArea / float(boxAArea + boxBArea - interArea)
	return iou

# combine output (rows) of OpenFace and ActiveSpeakerDetection, only if their detected faces have intersected bounding box, the threshold of IOU is 0.1
# ATTENTION: the output exclude frames where IOU<threshold, so the results are probably not continuous, currently we do not require continuous samples for training.
def combine(face, asd, x_frame, x_id=None, threshold=0.1, success_only=False):
    temp=[]
#     asd_dummy = [np.NaN]*len(asd.columns)
#     face_dummy = [np.NaN]*len(face.columns)
#     print(len(x_frame))
#     print(x_frame)
    for i in x_frame:
        df_face = face.loc[face['frame']==i]
        df_asd = asd.loc[asd['frame']==i]
        if x_id is not None:
            df_asd = df_asd.loc[df_asd['track_id']==x_id]
        if success_only:
            df_face = df_face.loc[df_face['success']==1]
        if (not df_face.empty) and (not df_asd.empty):
            # combine df_face and df_asd
            IOU = np.empty((len(df_face)*2,len(df_asd)*2)); IOU.fill(threshold)
            # *2 is for creating dummy rows and columns, but I am not sure if it's necessary
            bboxA = df_face[['xmin', 'ymin', 'xmax', 'ymax']]
            bboxB = df_asd[['bbox_xmin', 'bbox_ymin', 'bbox_xmax', 'bbox_ymax']]
            for j in range(len(df_face)): # row
                IOU[j][:len(df_asd)] = [bb_intersection_over_union(list(bboxA.iloc[j]), list(bboxB.iloc[k])) for k in range(len(df_asd))]
            row_ind, col_ind = linear_sum_assignment(-IOU)
            # row_ind, col_ind are something like [0,1,2] [1,2,0], where it means 0th of df_face paired with 1th of df_asd, 1th with 2th, and 2th with 0th
            # in other words, idx=0, 0--1, idx=1, 1--2, idx=2, 2--0
            # then we want to choose from above pairs, if the left--right satisfies left<len(df_face) and right<len(df_asd) (i.e. both left and right are not dummy)   
            # combine df_face and df_asd by row_ind, col_ind
            idx=0  
            for j in range(max(len(df_asd),len(df_face))): 
                if idx < len(col_ind) and col_ind[idx] < len(df_asd) and idx < len(row_ind) and row_ind[idx] < len(df_face):
                    left = list(df_asd.iloc[col_ind[idx]])
                    right = list(df_face.iloc[row_ind[idx]])
                    temp.append(left + right[1:])                
                idx+=1
#     print("len %d -> %d" % (len(x_frame), len(temp)))  
    return temp

In [11]:
folder2 = [Path(".\\openface_data\\openface_features\\train"),Path(".\\openface_data\\openface_features\\test"),Path(".\\openface_data\\openface_features\\val")]
subfolder2 = []
for i in folder2:
    temp=[i / f for f in os.listdir(i) if not f.startswith('.') and os.path.isdir(i / f)]
    subfolder2 = subfolder2 + temp
# assert len(subfolder)==len(subfolder2)

# just want to test on a single file
# subfolder2=subfolder2[-3:-2]
# print(subfolder2)

cbfolder=[]
for i in range(len(subfolder)):
    assert str(subfolder[i])[-9:]==str(subfolder2[i])[-9:]
    cbfolder.append((subfolder[i],subfolder2[i]))

In [13]:
# very slow
PP4 = True
input_file = "turn.json"
output_file = "training_sample_detail.json"
asd_file = "ASDout.csv"
data_folder = "data1"
# output (x_id, x_start, x_end, y_id, y_start, y_end), which is necessary information for a training sample

basic = ['frame','face_id','success']
gaze_direction = ['gaze_angle_x','gaze_angle_y']
# head_position = ['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz'] # first three can be removed
face_landmark2d = [] # this can be removed
for i in range(0,68):
    face_landmark2d.append("x_"+str(i))
for i in range(0,68):
    face_landmark2d.append("y_"+str(i))
PDM_rigid = ['p_scale', 'p_rx', 'p_ry', 'p_rz', 'p_tx', 'p_ty'] # can be removed
PDM_nonrigid = []
for i in range(0,34): # can be removed
    PDM_nonrigid.append("p_"+str(i))
FAU = ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r']
col_list = basic + gaze_direction + face_landmark2d + PDM_rigid + PDM_nonrigid + FAU
output_col = ['frame']+col_list[3:]
    
if PP4:
    for i,j in cbfolder:
        print(i,j)
        stat=[]
        face = pd.read_csv(j / (j.name+'.csv'), usecols=col_list)     
        
        x1,x2,y1,y2=face.columns.get_loc("x_0"),face.columns.get_loc("x_67"),face.columns.get_loc("y_0"),face.columns.get_loc("y_67")
        xmin,xmax,ymin,ymax=np.min(face.iloc[:,x1:x2],axis=1),np.max(face.iloc[:,x1:x2],axis=1),np.min(face.iloc[:,y1:y2],axis=1),np.max(face.iloc[:,y1:y2],axis=1)
        face['xmin'], face['ymin'], face['xmax'], face['ymax'] = [xmin,ymin,xmax,ymax]
        
        asd = pd.read_csv(i / asd_file)
        
        columns = list(asd.columns)+list(face.columns)[1:] 
        ind = [columns.index(i) for i in output_col]
        
        with open(i / input_file, "r") as fp:
            samples = json.load(fp)  # [x_id, x_frame, y_id], id is related to ASD 
                   
        Path(i / data_folder).mkdir(exist_ok=True)
        for k in range(len(samples)):
#         for k in range(1):
            x_id, x_frame, y_id = samples[k]["x_id"], samples[k]["x_frame"], samples[k]["y_id"]
            result = np.array(combine(face, asd, x_frame, x_id=x_id))
            if result.size>0:              
#                 print(result[:10,:])
#                 break
                stat.append({"filename":"data_"+str(k), "x_id":x_id, "y_id":y_id, 
                        "x_frame":result[:,0].tolist(), "x_pos":result[:,4:8].tolist()})
                data = np.float32(result[:,ind])
                output=(x_id, y_id, data)
                with open(i / data_folder / ("data_"+str(k)), "wb") as fp:
                    pickle.dump(output, fp)
        with open(i / output_file, 'w') as fp:
            json.dump(stat, fp)      
        print("finish "+str(i))      
#         break

asd_result\train\train_vimeo_410316487 openface_data\openface_features\train\train_vimeo_410316487
finish asd_result\train\train_vimeo_410316487
asd_result\train\train_vimeo_420368971 openface_data\openface_features\train\train_vimeo_420368971
finish asd_result\train\train_vimeo_420368971
asd_result\train\train_vimeo_429747058 openface_data\openface_features\train\train_vimeo_429747058
finish asd_result\train\train_vimeo_429747058
asd_result\train\train_vimeo_440357459 openface_data\openface_features\train\train_vimeo_440357459
finish asd_result\train\train_vimeo_440357459
asd_result\train\train_vimeo_460704945 openface_data\openface_features\train\train_vimeo_460704945
finish asd_result\train\train_vimeo_460704945
asd_result\train\train_vimeo_471034832 openface_data\openface_features\train\train_vimeo_471034832
finish asd_result\train\train_vimeo_471034832
asd_result\train\train_vimeo_480532944 openface_data\openface_features\train\train_vimeo_480532944
finish asd_result\train\train_v